# Final Project -- Audio Steganography & MFCC Steganalysis

## Amanda Foster

In [492]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import wave
from scipy.signal import butter, filtfilt
import pandas as pd

In [493]:
def insert_data(audio_file, message, output_file):
    try: 
      with wave.open(audio_file, 'rb') as file:
          frame_bytes = bytearray(file.readframes(file.getnframes()))
          sr = file.getframerate()
      
      message += "#####"
      message_bits = ''.join(format(ord(char), '08b') for char in message)
      message_length = len(message_bits)
      
      if message_length * 2 > len(frame_bytes):
          raise ValueError("Message is too large to be embedded in the audio file.")
      
      for i in range(message_length):
          frame_bytes[i] &= 0xFE
          frame_bytes[i] |= int(message_bits[i])
      
      with wave.open(output_file, 'wb') as file:
          file.setparams((1, 2, sr, len(frame_bytes), 'NONE', 'not compressed'))
          file.writeframes(frame_bytes)
      
    except Exception as e:
      print("An error occured while embedding the file: ", e)


def encode(audio_file, message):
  output_file = os.path.join("output_files", os.path.splitext(os.path.basename(audio_file))[0] + "_embedded")
  output_file += ".wav"
  
  insert_data(audio_file, message, output_file)

In [494]:
def extract_data(audio_file):
  try:
    with wave.open(audio_file, 'rb') as file:
        frame_bytes = bytearray(file.readframes(file.getnframes()))
    
    extracted_bits = []
    for byte in frame_bytes:
        extracted_bit = byte & 1
        extracted_bits.append(extracted_bit)
    
    extracted_message = ''.join(chr(int(''.join(map(str, extracted_bits[i:i+8])), 2)) for i in range(0, len(extracted_bits), 8))
    
    end_index = extracted_message.find('#####')
    if end_index != -1:
        extracted_message = extracted_message[:end_index]
    
    return extracted_message
  
  except Exception as e:
    print("An error occurred while extracting the message", e)
    return None
  
def decode(audio_file):
  extracted_message = extract_data(audio_file)
  return extracted_message

In [495]:
def compute_entropy(signal):
  hist, _ = np.histogram(signal, bins=256, range=(0,255))
  probs = hist / np.sum(hist)
  entropy = -np.sum(probs * np.log2(probs + 1e-10))
  return entropy

In [496]:
def extract_features(audio_file):
  num_mfccs = 60
  y, sr = librosa.load(audio_file)
  mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_mfccs)
  mfccs = np.mean(mfccs, axis=0)
  
  var = np.var(y)
  cov = float(np.cov(y))
  mean = np.mean(y)
  median = np.median(y)
  entropy = compute_entropy(y)

  mfccs_list = [mfccs[i] for i in range(0,num_mfccs)]  
  features = mfccs_list + [var, cov, mean, median, entropy]
  
  return features

## Reading Input Files and Embedding Messages

In [497]:
input_files_dir = "input_files"
output_files_dir = "output_files"
extracted_messages = []

input_data = []
input_labels = []
input_filenames = []

for filename in os.listdir(input_files_dir):
    if filename.endswith(".wav"):
        audio_file = os.path.join(input_files_dir, filename)
        message = f'we embedded a message into {os.path.splitext(os.path.basename(audio_file))[0]}.wav'
        encode(audio_file, message)
        
        embedded_audio_file = os.path.join(output_files_dir, os.path.splitext(os.path.basename(audio_file))[0] + "_embedded.wav")
        extracted_message = decode(embedded_audio_file)
        
        extracted_messages.append(extracted_message)
        
        features = extract_features(audio_file)
        input_data.append(features)
        input_labels.append(0)
        input_filenames.append(filename)

print("Extracted messages:", extracted_messages[0:5])

Extracted messages: ['we embedded a message into LJ003-0252.wav', 'we embedded a message into LJ001-0118.wav', 'we embedded a message into LJ003-0300.wav', 'we embedded a message into LJ001-0016.wav', 'we embedded a message into LJ002-0258.wav']


## Reading Output Files

In [498]:
output_data = []
output_labels = []
output_filenames = []
output_correct = []

for filename in os.listdir(output_files_dir):
    if filename.endswith('.wav'):
        audio_file = os.path.join(output_files_dir, filename)
        features = extract_features(audio_file)
        output_data.append(features)
        output_labels.append(1)
        output_filenames.append(filename)
        output_correct.append(decode(audio_file) == f'we embedded a message into {os.path.splitext(os.path.basename(audio_file))[0].split("_")[0]}.wav')

In [499]:
input_df = pd.DataFrame({
    'filename': input_filenames,
    'features': input_data,
    'label': input_labels,
})

output_df = pd.DataFrame({
    'filename': output_filenames,
    'features': output_data,
    'label': output_labels,
    'output_correct': output_correct
})

df = pd.concat([input_df, output_df], ignore_index=True)

df.head(-6)

,filename,features,label,output_correct
0,LJ003-0252.wav,"[-7.4575434, -6.7175727, -6.6358967, -6.94033,...",0,NaN
1,LJ001-0118.wav,"[-6.7034698, -5.5601625, -5.975423, -6.106264,...",0,NaN
2,LJ003-0300.wav,"[-7.9909806, -7.0308933, -5.8685746, -4.882595...",0,NaN
3,LJ001-0016.wav,"[-7.0968084, -5.983182, -6.495754, -7.5794964,...",0,NaN
4,LJ002-0258.wav,"[-6.710439, -5.975958, -6.0198135, -5.640287, ...",0,NaN
...,...,...,...,...
1637,LJ003-0183_embedded.wav,"[-6.113185, -6.6125593, -8.641822, -8.114795, ...",1,True
1638,LJ002-0258_embedded.wav,"[-5.056022, -5.4978404, -6.0077496, -5.640287,...",1,True
1639,LJ003-0061_embedded.wav,"[-5.9386363, -6.567407, -7.933168, -7.339717, ...",1,True
1640,LJ003-0264_embedded.wav,"[-5.482971, -5.7169714, -7.3388376, -7.5587873...",1,True


In [500]:
output_files = df[df['label'] == 1]
incorrect_messages = output_files[output_files['output_correct'] == False]

print("Number of incorrect messages:", len(incorrect_messages))

Number of incorrect messages: 0



# Detecting with ML Techniques

In [501]:
X = np.array(df['features'].tolist())
y = df['label'].values

### SVM

In [502]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lda = LinearDiscriminantAnalysis()
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

clf = SVC(kernel='rbf', gamma='scale')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8878787878787879


### Random Forests

In [503]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lda = LinearDiscriminantAnalysis()
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

clf = RandomForestClassifier(n_estimators=10, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.8848484848484849
